In [2]:
import segyio
import os
import numpy as np

In [63]:
s_path = os.path.join("..", "data", "NS2950-2180_3000-2230.sgy")
s_path1 = os.path.join("..", "data", "MS2950-2180_3000-2230.sgy")
s_path2 = os.path.join("..", "data", "FS2950-2180_3000-2230 (copia).sgy")
new_seismic = os.path.join("..", "data", "NMF2950-2180_3000-2230.sgy")
smallps = os.path.join("..", "data", "small-ps.sgy")
gather_path = [s_path, s_path1, s_path2]

angle_bet_gathers = 12

In [64]:
    print(f.iline[1,1])
    print(" ")
    print(f.iline[1,2])
    print(" ")
    print(f.iline[1])
    print(" ")
    print(f.offsets)
    print(f.trace[0])

KeyError: 1

In [98]:
with segyio.open(smallps, "r") as f:
    print(f.header[4])
    print(f.ilines)

{TRACE_SEQUENCE_LINE: 0, TRACE_SEQUENCE_FILE: 0, FieldRecord: 0, TraceNumber: 0, EnergySourcePoint: 0, CDP: 0, CDP_TRACE: 0, TraceIdentificationCode: 0, NSummedTraces: 0, NStackedTraces: 0, DataUse: 0, offset: 1, ReceiverGroupElevation: 0, SourceSurfaceElevation: 0, SourceDepth: 0, ReceiverDatumElevation: 0, SourceDatumElevation: 0, SourceWaterDepth: 0, GroupWaterDepth: 0, ElevationScalar: 0, SourceGroupScalar: 0, SourceX: 0, SourceY: 0, GroupX: 0, GroupY: 0, CoordinateUnits: 0, WeatheringVelocity: 0, SubWeatheringVelocity: 0, SourceUpholeTime: 0, GroupUpholeTime: 0, SourceStaticCorrection: 0, GroupStaticCorrection: 0, TotalStaticApplied: 0, LagTimeA: 0, LagTimeB: 0, DelayRecordingTime: 0, MuteTimeStart: 0, MuteTimeEND: 0, TRACE_SAMPLE_COUNT: 0, TRACE_SAMPLE_INTERVAL: 0, GainType: 0, InstrumentGainConstant: 0, InstrumentInitialGain: 0, Correlated: 0, SweepFrequencyStart: 0, SweepFrequencyEnd: 0, SweepLength: 0, SweepType: 0, SweepTraceTaperLengthStart: 0, SweepTraceTaperLengthEnd: 0, T

# Merging sgys

In [66]:
from shutil import copyfile

In [67]:
        #s.header.iline[:] = {37: 12}
        #s.header.iline[:,24] = {37: 24}
        #s.header.iline[:,36] = {37: 36}
        #s.header.offset = offsts
        #print(s.header[0])

# change the offset in the gathers
to change the offser header then to merge to see what the fuck is happening

In [68]:
for file in gather_path:
    with segyio.open(file, "r+") as f:
        print(f.header[0][37])

0
0
0


In [136]:
# source1: https://github.com/equinor/segyio/blob/master/python/examples/make-file.py
# source2: https://github.com/equinor/segyio/blob/master/python/segyio/tracefield.py

# declaring offsets to set
offsts = np.array([12, 24, 36])

# Initializing files
with segyio.open(s_path) as f, segyio.open(s_path1) as g, segyio.open(s_path2) as h:
    spec = segyio.spec()
    spec.sorting = f.sorting
    spec.format = f.format
    spec.samples = f.samples
    spec.ilines = f.ilines
    spec.xlines = f.xlines
    spec.offsets = offsts
    
    # Initializing the merge
    with segyio.create(new_seismic, spec) as s:
        
        # Trace in the merge
        tr = 0
        # Trace in the original
        tr1 = 0
        # For loop to set parameters according to the seismic lines and offset
        for il in spec.ilines:
            for xl in spec.xlines:
                for offset in spec.offsets:
                    # Assigning headers [byte]
                    s.header[tr] = {segyio.su.tracl  : f.header[tr1][1],
                                    segyio.su.tracr  : f.header[tr1][5],
                                    segyio.su.fldr   : f.header[tr1][9],
                                    segyio.su.cdp    : f.header[tr1][21],
                                    segyio.su.cdpt   : f.header[tr1][25],
                                    segyio.su.offset : offset, # 37
                                    segyio.su.scalco : f.header[tr1][71],
                                    segyio.su.ns     : f.header[tr1][115],
                                    segyio.su.dt     : f.header[tr1][117],
                                    segyio.su.cdpx   : f.header[tr1][181],
                                    segyio.su.cdpy   : f.header[tr1][185],
                                    segyio.su.iline  : il,  # 189
                                    segyio.su.xline  : xl}  # 193
                    s.trace[tr] = f.trace[tr1]
                    tr += 1
                tr1 += 1

In [142]:
with segyio.open(new_seismic, "r") as f, segyio.open(s_path) as g:
    #print(f.header[0])
    #print(g.header[0][9])
    #print(g.header[0])
    print(f.offsets)
    print(f.gather[2950, 2180,12])

[12 24 36]
[0. 0. 0. ... 0. 0. 0.]


In [44]:
offsts = np.array([12])
with segyio.open(s_path) as f, segyio.open(s_path1) as g, segyio.open(s_path2) as h:
    spec = segyio.spec()
    spec.sorting = f.sorting
    spec.format = f.format
    spec.samples = f.samples
    # CALCULATED THROUGH THE OFFSET ARRAY
    #spec.tracecount = f.tracecount
    spec.ilines = f.ilines
    spec.xlines = f.xlines
    spec.offsets = offsts
    with segyio.create(new_seismic, spec) as s:
        print(s.header[5201])
        print(s.tracecount)
        
        # Testing things
        
        # if the data is prestack, f.offsets should return an np.array != 0
        #print(s.offsets)
        # if the data is prestack, then the header[0][37] != 0 // header[0][37] == 3
        #print(s.header[0][37] == 3)
        # Trace[0] should give the first trace the function finds... theorically it's f.iline[2950,12]
        #print(s.trace[0][750] == s.iline[2950,12][0][750])


{TRACE_SEQUENCE_LINE: 0, TRACE_SEQUENCE_FILE: 0, FieldRecord: 0, TraceNumber: 0, EnergySourcePoint: 0, CDP: 0, CDP_TRACE: 0, TraceIdentificationCode: 0, NSummedTraces: 0, NStackedTraces: 0, DataUse: 0, offset: 0, ReceiverGroupElevation: 0, SourceSurfaceElevation: 0, SourceDepth: 0, ReceiverDatumElevation: 0, SourceDatumElevation: 0, SourceWaterDepth: 0, GroupWaterDepth: 0, ElevationScalar: 0, SourceGroupScalar: 0, SourceX: 0, SourceY: 0, GroupX: 0, GroupY: 0, CoordinateUnits: 0, WeatheringVelocity: 0, SubWeatheringVelocity: 0, SourceUpholeTime: 0, GroupUpholeTime: 0, SourceStaticCorrection: 0, GroupStaticCorrection: 0, TotalStaticApplied: 0, LagTimeA: 0, LagTimeB: 0, DelayRecordingTime: 0, MuteTimeStart: 0, MuteTimeEND: 0, TRACE_SAMPLE_COUNT: 0, TRACE_SAMPLE_INTERVAL: 0, GainType: 0, InstrumentGainConstant: 0, InstrumentInitialGain: 0, Correlated: 0, SweepFrequencyStart: 0, SweepFrequencyEnd: 0, SweepLength: 0, SweepType: 0, SweepTraceTaperLengthStart: 0, SweepTraceTaperLengthEnd: 0, T

In [46]:
with segyio.open(new_seismic, "r") as f:
    print(f.header[0])

OSError: I/O operation failed on trace header 0, likely corrupted file

In [82]:
with segyio.open(s_path) as segy, segyio.open(s_path1) as segy1, segyio.open(s_path2) as segy2:
    spec = segyio.spec()
    spec.sorting = segy.sorting
    spec.format = segy.format
    spec.samples = segy.samples
    spec.ilines = segy.ilines
    spec.xlines = segy.xlines
    spec.offsets[:] = np.array([12, 24, 36])
    with segyio.create(new_seismic, spec) as nsegy:
        nsegy.text[0] = segy.text[0]
        nsegy.bin = segy.bin
        nsegy.header = segy.header
        nsegy.header.iline[1:3, 2] = {1: 13}

        nsegy.iline[:,12] = segy.iline[:]
        nsegy.iline[:,24] = segy1.iline[:]
        nsegy.iline[:,36] = segy2.iline[:]

        

In [83]:
with segyio.open(new_seismic, spec) as nsegy:
    print(nsegy.header[1])

TypeError: argument of type 'spec' is not iterable

In [40]:
with segyio.open(s_path) as segy, segyio.open(s_path1) as segy1, segyio.open(s_path2) as segy2:
    spec = segyio.spec()
    spec.sorting = segy.sorting
    spec.format = segy.format
    spec.samples = segy.samples
    spec.ilines = segy.ilines
    spec.xlines = segy.xlines
    spec.offsets[:] = np.array([12, 24, 36])
    with segyio.create(new_seismic, spec) as nsegy:
        nsegy.text[0] = segy.text[0]
        nsegy.bin = segy.bin
        nsegy.header = segy.header
        nsegy.header.offset = np.array([12, 24, 36])
        

In [41]:
with segyio.open(new_seismic, "r") as nsegy:
    print(nsegy.offsets)

RuntimeError: trace count inconsistent with file size, trace lengths possibly of non-uniform

In [44]:
with segyio.open(new_seismic, "r", spec) as nsegy:
    print(nsegy.offsets)

RuntimeError: trace count inconsistent with file size, trace lengths possibly of non-uniform

In [125]:
np.set_printoptions(threshold=np.inf)
with segyio.open(new_seismic, "r+") as seg:
    seg.header[0][34]
    #seg.attributes(segyio.TraceField.offset)[0] = np.array([12,24,36])
    #print(seg.attributes(segyio.TraceField.offset)[0])
    #print(type(seg.attributes(segyio.TraceField.offset)[0]))


KeyError: 'No such field 2'

In [62]:
np.zeros([1,1501])

array([[0., 0., 0., ..., 0., 0., 0.]])

In [74]:
np.arange(100)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])